# Word Embeddings : le modèle Word2Vec de la décennie 1880 et 1960

## Imports

In [23]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents2_tp4.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

In [4]:
bigram_phrases = Phrases(sentences)

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

607136

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

rue


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

7114

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [9]:
%time bigram_phrases[sentences[78]]

CPU times: user 64 µs, sys: 4 µs, total: 68 µs
Wall time: 73.7 µs


['restauration_eventuelle', 'de', 'certaines_parties', 'de', 'facades', 'en']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [11]:
%time bigram_phraser[sentences[78]]

CPU times: user 33 µs, sys: 2 µs, total: 35 µs
Wall time: 39.8 µs


['restauration_eventuelle', 'de', 'certaines_parties', 'de', 'facades', 'en']

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['art', '.'], ['74', '.'], ['--', 'subside', 'a', 'vathenee', 'royal', ':', '82', ',', '000_francs', '.'], ['cette_somme', 'represente', 'la', 'part', 'incombant', 'a', 'ia', 'ville', 'dans'], ['le', 'traitement', 'des', 'professeurs', ',', 'ainsi', 'que', 'le', 'montant', 'des'], ['menues', 'depenses', '.'], ['le', 'budget', 'presente', 'par', 'la', 'commission_administrative'], ['ainsi', 'que', 'le', 'compte', 'justifiant', 'l', "'", 'emploi', 'de', 'ces', 'fonds', ',', 'sont'], ['annuellement', 'soumis', 'a', 'l', "'", 'approbation', 'du', 'conseil_communal', '.'], ['art', '.'], ['75', '.'], ['--', 'loyer', 'de', 'locaux', ',', 'entretien', ',', 'contributions', 'et', 'assurance', ':'], ['29', ',', '750', 'francs', '.'], ['loyer', 'des', 'locaux', 'de', 'la', 'section_professionnelle', 'de', '1', 'athenee', ',', 'rue', 'du'], ['fr', '.'], ['19', ',', '700'], ['grand', '-', 'hospice'], ['.'], ['.'], ['.'], ['.'], ['.'], ['1', ',', '050'], ['contributions', 'et', 'assurance', '.'], [

## Entraînement d'un modèle Word2Vec sur ce corpus

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 31.9 s, sys: 158 ms, total: 32.1 s
Wall time: 17 s


### Sauvegarder le modèle dans un fichier

In [17]:
outfile = f"../data/bulletins_tp4.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [18]:
model = Word2Vec.load("../data/bulletins_tp4.model")

### Imprimer le vecteur d'un terme

In [20]:
model.wv["art"]

array([-0.7086632 , -0.33289444,  2.1562    ,  0.5013727 , -0.73077095,
       -0.76644933, -2.7560346 ,  0.38221553,  1.2058228 ,  0.86362714,
       -3.3544064 , -2.294019  , -2.6045032 , -0.53698164,  1.1983888 ,
        0.51813453,  0.5891617 ,  1.3009491 ,  0.37372476, -0.35540214,
        2.1480296 ,  0.31965113, -2.4387014 , -2.556127  ,  0.4505468 ,
        0.7742366 , -1.3374887 , -0.7420905 , -1.2654073 ,  2.577202  ,
        0.41294035, -1.2392157 ], dtype=float32)

### Calculer la similarité entre deux termes

In [21]:
model.wv.similarity("art", "moderne")

0.016951732

### Chercher les mots les plus proches d'un terme donné

In [22]:
model.wv.most_similar("art", topn=10)

[('article_premier', 0.7752205729484558),
 ('autorite_superieure', 0.715242862701416),
 ('exercice', 0.6989240050315857),
 ('administration_charitable', 0.6899999380111694),
 ('article', 0.6881192922592163),
 ('exercice_1888', 0.6836838722229004),
 ('exercice_1961', 0.6719928979873657),
 ('extraordinaire', 0.6692975759506226),
 ('hopital_brugmann', 0.6494511961936951),
 ('etranger', 0.6475836038589478)]

### Faire des recherches complexes à travers l'espace vectoriel

In [25]:
print(model.wv.most_similar(positive=['art', 'etranger'], negative=['belgique']))

[('echevin_vanden_boeynants', 0.7327759861946106), ('echevin_piron', 0.7135235071182251), ('echevin_morelle', 0.7042292356491089), ('honorable_membre', 0.6943852305412292), ('unanimite', 0.6939157247543335), ('administration_charitable', 0.6901442408561707), ('heure_actuelle', 0.6817639470100403), ('echevin_andre', 0.6673774719238281), ('honorable', 0.6552387475967407), ('annee_derniere', 0.6543301939964294)]


# Word Embeddings : le modèle Word2Vec de la décennie 1880

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents_1880_tp4.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

In [4]:
bigram_phrases = Phrases(sentences)

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

650328

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

rue


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

8679

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [9]:
%time bigram_phrases[sentences[78]]

CPU times: user 94 µs, sys: 6 µs, total: 100 µs
Wall time: 110 µs


['restauration_eventuelle', 'de', 'certaines_parties', 'de', 'facades', 'en']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [11]:
%time bigram_phraser[sentences[78]]

CPU times: user 37 µs, sys: 0 ns, total: 37 µs
Wall time: 40.8 µs


['restauration_eventuelle', 'de', 'certaines_parties', 'de', 'facades', 'en']

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['art', '.'], ['74', '.'], ['--', 'subside', 'a', 'vathenee', 'royal', ':', '82', ',', '000_francs', '.'], ['cette', 'somme_represente', 'la', 'part_incombant', 'a', 'ia', 'ville', 'dans'], ['le', 'traitement', 'des', 'professeurs', ',', 'ainsi', 'que', 'le', 'montant', 'des'], ['menues_depenses', '.'], ['le', 'budget', 'presente', 'par', 'la', 'commission_administrative'], ['ainsi', 'que', 'le', 'compte', 'justifiant', 'l', "'", 'emploi', 'de', 'ces', 'fonds', ',', 'sont'], ['annuellement', 'soumis', 'a', 'l', "'", 'approbation', 'du', 'conseil_communal', '.'], ['art', '.'], ['75', '.'], ['--', 'loyer', 'de', 'locaux', ',', 'entretien', ',', 'contributions', 'et', 'assurance', ':'], ['29', ',', '750', 'francs', '.'], ['loyer', 'des', 'locaux', 'de', 'la', 'section_professionnelle', 'de', '1', 'athenee', ',', 'rue', 'du'], ['fr', '.'], ['19', ',', '700'], ['grand', '-', 'hospice'], ['.'], ['.'], ['.'], ['.'], ['.'], ['1', ',', '050'], ['contributions', 'et', 'assurance', '.'], ['.'], 

## Entraînement d'un modèle Word2Vec sur ce corpus

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 41.3 s, sys: 215 ms, total: 41.5 s
Wall time: 21.2 s


### Sauvegarder le modèle dans un fichier

In [17]:
outfile = f"../data/bulletins_1880_tp4.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [18]:
model = Word2Vec.load("../data/bulletins_1880_tp4.model")

### Imprimer le vecteur d'un terme

In [19]:
model.wv["art"]

array([ 0.17087047, -2.551908  ,  0.78166103,  0.9206635 ,  1.7238601 ,
        0.85198015,  2.439297  , -1.1999629 ,  1.1384604 , -0.35282376,
        0.47259858, -0.18319729, -0.19545233,  3.3343716 ,  0.634813  ,
       -0.04328004, -4.195551  , -1.7860304 , -2.7397084 ,  0.59589237,
       -3.7685313 ,  0.23774858,  1.3128002 ,  1.6910052 ,  3.8838584 ,
       -3.397279  , -2.2818036 ,  2.9174223 , -0.15669665,  0.49481836,
       -1.9808458 ,  2.1420782 ], dtype=float32)

### Calculer la similarité entre deux termes

In [20]:
model.wv.similarity("art", "moderne")

0.017246462

### Chercher les mots les plus proches d'un terme donné

In [21]:
model.wv.most_similar("art", topn=10)

[('article', 0.7527716159820557),
 ('arrete_organique', 0.6500847339630127),
 ('annee_derniere', 0.6386935114860535),
 ('incident_est_clos', 0.6297671794891357),
 ('autorite_superieure', 0.62404465675354),
 ('exercice', 0.6240027546882629),
 ('exercice_1888', 0.6220220327377319),
 ('exercice_1887', 0.6206351518630981),
 ('enseignement_primaire', 0.6098679304122925),
 ('honorable_echevin_du_contentieux', 0.604128360748291)]

### Faire des recherches complexes à travers l'espace vectoriel

In [22]:
print(model.wv.most_similar(positive=['art', 'etranger'], negative=['belgique']))

[('echevin_andre', 0.6900877952575684), ('octroi', 0.6778690814971924), ('echevin_delecosse', 0.6682404279708862), ('allocation_budgetaire', 0.6629053354263306), ('echevin_janssen', 0.6605156660079956), ('enseignement_moyen', 0.6584456562995911), ('article', 0.6581102609634399), ('echevin_buis', 0.6506021022796631), ('orphelinat', 0.6486164331436157), ('administration_centrale', 0.6438518166542053)]


# Word Embeddings : le modèle Word2Vec de la décennie 1960

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [24]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [25]:
infile = f"../data/sents_1960_tp4.txt"
sentences = MySentences(infile)

sentences = [sentence for sentence in sentences]

### Détection des bigrams

In [26]:
bigram_phrases = Phrases(sentences)

In [27]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [28]:
len(bigram_phrases.vocab.keys())

544882

Prenons une clé au hasard :

In [29]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

e_.


Le dictionnaire indique le score de cette coocurrence :

In [30]:
bigram_phrases.vocab[key_]

4627

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [31]:
%time bigram_phrases[sentences[78]]

CPU times: user 45 µs, sys: 1 µs, total: 46 µs
Wall time: 50.8 µs


['molenbeek', ',', 'rue_blaes', ',']

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [32]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [33]:
%time bigram_phraser[sentences[78]]

CPU times: user 28 µs, sys: 0 ns, total: 28 µs
Wall time: 31.9 µs


['molenbeek', ',', 'rue_blaes', ',']

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [34]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [35]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [36]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [37]:
print(corpus[:100])

[['--'], ['301'], ['--'], ['5', '.'], ['--', 's', 'e', 'r', 'v', 'i', 'c', 'e', 'a', 'd', 'm', 'i', 'n', 'i', 's', 't', 'r', 'a', 't', 'i', 'f'], ['51', '.'], ['--'], ['administration', '.'], ['d', 'u', 'r', 'a', 'n', 't', 'l', "'", 'annee', '1960', ',', 'i', 'l', 'a', 'ete_procede'], ['1', '3', '.'], ['3', '9', '9', 'pieces', 'a', 'l', "'", 'i', 'n', 'd', 'i', 'c', 'a', 't', 'e', 'u', 'r', '.'], ['a', 'l', "'", 'inscription', 'de'], ['i', 'l', 'a', 'ete_procede', 'a', '3', '3', 'adjudications_publiques', ',', '107', 'adjudications_restreintes', ',', 'ainsi', 'q_u', "'", 'a', '531', 'demandes', 'de', 'prix', '.'], ['d', "'", 'a', 'u', 't', 'r', 'e', 'part', ',', 'i', 'l', 'a', 'ete_emis', '1', '.', '551', 'bons', 'de', 'commande', 'au', 'secteur_prive', 'et', '197', 'bons', 'de', 's', 'e', 'r', 'v', 'i', 'c', 'e', '.'], ['il', 'a', 'e', 't', 'e', 'o', 'u', 'v', 'e', 'r', 't', '134', 'dossiers', 'relatifs', 'a', 'des', 'accidents', 'interessant', 'le', 's', 'e', 'r', 'v', 'i', 'c', 'e',

## Entrainement d'un modèle Word2Vec sur ce corpus

In [38]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 31.7 s, sys: 180 ms, total: 31.9 s
Wall time: 17.2 s


### Sauvegarder le modèle dans un fichier

In [39]:
outfile = f"../data/bulletins_1960_tp4.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [40]:
model = Word2Vec.load("../data/bulletins_1960_tp4.model")

### Imprimer le vecteur d'un terme

In [41]:
model.wv["art"]

array([-2.6597617 , -1.241102  , -0.27512133,  0.69093704, -0.30084085,
        0.26886144, -1.3192028 , -0.5619176 ,  0.59281737, -0.48175868,
       -2.026145  , -1.114153  , -1.3347768 ,  0.28322062,  0.28051662,
       -0.27948004,  0.6806517 , -0.54023886, -0.93341136,  2.771631  ,
       -0.11169828,  1.3298135 , -0.931539  , -0.77403677,  1.2504743 ,
        0.21471673, -4.509209  , -0.71215755,  0.62367254, -0.2478028 ,
       -0.2937888 ,  0.96747744], dtype=float32)

### Calculer la similarité entre deux termes

In [42]:
model.wv.similarity("art", "moderne")

0.010911413

### Chercher les mots les plus proches d'un terme donné

In [43]:
model.wv.most_similar("art", topn=10)

[('etabli_selon', 0.6753695011138916),
 ('auteur_.)', 0.6619933843612671),
 ('autori', 0.6449148058891296),
 ('in_miljoene', 0.6109183430671692),
 ('institut_bordet', 0.6031122803688049),
 ('paulette', 0.6023857593536377),
 ('equilibre_normal', 0.6007688641548157),
 ('(tm)),_professeur', 0.593051016330719),
 ('hopital_brugmann', 0.5928184986114502),
 ('soir_).', 0.5882861018180847)]

### Faire des recherches complexes à travers l'espace vectoriel

In [44]:
print(model.wv.most_similar(positive=['art', 'etranger'], negative=['belgique']))

[('echevin_vanden_boeynants', 0.667678952217102), ('athenee', 0.6507083177566528), ('echevin_van_leynseele', 0.647958517074585), ('exterieur', 0.6436848044395447), ('auteur_.)', 0.6366246342658997), ('echevin_van_halteren', 0.6340541243553162), ('autori', 0.6335548162460327), ('echevin_morelle', 0.6251212358474731), ('etabli_selon', 0.614898681640625), ('institut_bordet', 0.6016395092010498)]
